In [1]:
# Load dataset
from datasets import load_dataset

dataset = load_dataset("swahili")
dataset

Generating train split:   0%|          | 0/42069 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3371 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3372 [00:00<?, ? examples/s]

Dataset swahili downloaded and prepared to /home/ara/.cache/huggingface/datasets/swahili/swahili/1.0.0/15bf1d99abb939f83b5da3c798ed55e9803b3ea430f06bf7e003bd073b60172a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 42069
    })
    test: Dataset({
        features: ['text'],
        num_rows: 3371
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3372
    })
})

In [3]:
texts = dataset['train']['text'] + dataset['test']['text'] + dataset['validation']['text']
print(len(texts))

48812


In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("eolang/SW-v1")
model = AutoModelForMaskedLM.from_pretrained("eolang/SW-v1")

In [5]:
# text: "Tumefanya mabadiliko muhimu [MASK] sera zetu za faragha na vidakuzi"

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)
sample_text = "Tumefanya mabadiliko muhimu [MASK]"

In [6]:
for prediction in fill_mask(sample_text):
    print(f"{prediction['sequence']}, confidence: {prediction['score']}")

tumefanya mabadiliko muhimu., confidence: 0.9650798439979553
tumefanya mabadiliko muhimu sana, confidence: 0.007741141598671675
tumefanya mabadiliko muhimu zaidi, confidence: 0.005796852055937052
tumefanya mabadiliko muhimu kwa, confidence: 0.0012140112230554223
tumefanya mabadiliko muhimu!, confidence: 0.0010500372154638171


In [10]:
predictions = fill_mask(sample_text)
predictions[0]

{'score': 0.9650798439979553,
 'token': 19,
 'token_str': '.',
 'sequence': 'tumefanya mabadiliko muhimu.'}

In [20]:
def next_token(sequence, num_of_items = 3):
    num_of_itterations = 0
    text = str(sequence)
    
    while num_of_itterations < num_of_items:
        mask_id = "[MASK]"
        text_seq = text + mask_id

        results = fill_mask(text_seq)
        #pred_token = results[0]['token_str']
        pred_sequence = results[1]['sequence']

        text = str(pred_sequence)
        print(text)

        num_of_itterations += 1

    return text


In [21]:
next_token('tumefanya mabadiliko')

tumefanya mabadiliko zaidi
tumefanya mabadiliko zaidi zaidi
tumefanya mabadiliko zaidi zaidi!


'tumefanya mabadiliko zaidi zaidi!'

In [24]:
next_token('sera zetu za faragha', 9)

sera zetu za faragha ni
sera zetu za faragha ni muhimu
sera zetu za faragha ni muhimu sana
sera zetu za faragha ni muhimu sana!
sera zetu za faragha ni muhimu sana!!
sera zetu za faragha ni muhimu sana!!!
sera zetu za faragha ni muhimu sana!!!?
sera zetu za faragha ni muhimu sana!!!??
sera zetu za faragha ni muhimu sana!!!??.


'sera zetu za faragha ni muhimu sana!!!??.'